- bookID: Unique identification number fro each book
- title: Name under which book was published
- authors: Name of the Authors of the book
- average_rating: Avarage rating of the book recevied in total.
- isbn: International standarded book number
- isbn13: 13 digit isbn to identify the book
- language_code: Primary Language of the book
- num_pages: Number of pages the book containes
- ratings_count: Total Number of ratings the book recevied.
- text_reviews_count: Total number of written reviews recevied.
- publication_date: Date when the book was first published
- publisher: Name of the Pulishers

In [ ]:
import pandas as pd
import numpy as np

# for data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import re



In [ ]:
df = pd.read_csv(r"C:\Users\user\OneDrive\diug\OneDrive\Desktop\Book-Recommendation-system\dataset\books.csv", on_bad_lines = 'skip')

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.describe(include = 'object')

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().any()

In [ ]:
df.info()

# Feature Engineering

- Extract Important Features
- Reducing the size of Features
- Creating new features from the existring ones

In [ ]:
df.columns

In [ ]:
df.isbn.nunique()



In [ ]:
df.isbn13.nunique()

In [ ]:
df.drop(['bookID', 'isbn', 'isbn13','text_reviews_count'], axis = 1, inplace = True)

In [ ]:
df.columns

In [ ]:
df[['publication_date']]

In [ ]:
df['year'] = df['publication_date'].str.split('/')
df['year'] = df['year'].apply(lambda x: x[2])

In [ ]:
df.head(2)

In [ ]:
df.dtypes

In [ ]:
df['year'] = df['year'].astype('int')

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df['year'].min()

In [ ]:
df['year'].max()

In [ ]:
df.columns

In [ ]:
df['authors'] = df['authors'].str.split('/').str[0]


In [ ]:
df.head(5)

# Exploratory Data Analysis

In [ ]:
df[df['year'] == 2020][['title', 'authors','average_rating','language_code','publisher' ]]

In [ ]:
df.groupby(['year'])['title'].agg('count').sort_values(ascending = False).head(10)

In [ ]:
df['year'].max()

In [ ]:
df.tail(5)

In [ ]:
plt.figure(figsize = (20, 10))
sns.countplot(x = 'authors', data = df,
             order = df['authors'].value_counts().iloc[:10].index,color='green')
plt.title("Top 10 Authors with maximum book publish")
plt.xticks(fontsize = 12)
plt.show()

In [ ]:
df.columns

In [ ]:
df.language_code.value_counts()

In [ ]:
df.groupby(['language_code'])[['average_rating', 
                               'ratings_count', 
                               ]].agg('mean').style.background_gradient(cmap = 'Wistia')


In [ ]:
book = df['title'].value_counts()[:20]
book

In [ ]:
# to find most occuring book in our data
plt.figure(figsize = (20, 6))
book = df['title'].value_counts()[:20]
sns.barplot(x = book.index, y = book,
           palette = 'winter_r')
plt.title("Most occuring Books")
plt.xlabel("Number of Occurance")
plt.ylabel("Books")
plt.xticks(rotation = 75, fontsize = 13)
plt.show()


# sns.barplot(...): This function creates a bar plot.
# x=book.index: Sets the x-axis to display the book titles (the index of the book Series).
# y=book: Sets the y-axis to display the counts of occurrences.
# palette='winter_r': Specifies a color palette for the bars. The 'winter_r' palette provides a gradient of colors from blue to green.

In [ ]:
sns.distplot(df['average_rating'])
plt.show()

In [ ]:
df[df.average_rating == df.average_rating.max()][['title','authors','language_code','publisher']]

In [ ]:
publisher = df['publisher'].value_counts()[:20]
publisher

In [ ]:
publisher = df['publisher'].value_counts()[:20]
sns.barplot(x = publisher.index, y = publisher, palette = 'winter_r')
plt.title("Publishers")
plt.xlabel("Number of Occurance")
plt.ylabel("Publishers")
plt.xticks(rotation = 75, fontsize = 12)
plt.show()

In [ ]:
# df.publisher.value_counts()
df[['publisher']].value_counts()

In [ ]:
df.to_csv(r'C:\Users\user\OneDrive\diug\OneDrive\Desktop\Book-Recommendation-system\dataset\cleaned_data.csv',index=False)


# Data Preprocessing

In [ ]:
df= pd.read_csv(r"C:\Users\user\OneDrive\diug\OneDrive\Desktop\Book-Recommendation-system\dataset\cleaned_data.csv")

In [ ]:
df.head(5)

In [ ]:
def num_to_obj(x):
    if x >0 and x <=1:
        return "between 0 and 1"
    if x > 1 and x <= 2:
        return "between 1 and 2"
    if x > 2 and x <=3:
        return "between 2 and 3"
    if x >3 and x<=4:
        return "between 3 and 4"
    if x >4 and x<=5:
        return "between 4 and 5"
df['rating_obj'] = df['average_rating'].apply(num_to_obj)

In [ ]:
df['rating_obj'].value_counts()

In [ ]:
rating_df = pd.get_dummies(df['rating_obj'])     #one hot encoding
rating_df.head()

In [ ]:
df.columns

In [ ]:
language_df = pd.get_dummies(df['language_code'])
language_df.head()

In [ ]:
df[df['publisher']=='Scholastic Inc.'].count()

In [ ]:
df[df['title']=='The Secret Garden']

In [ ]:
publisher_df=pd.get_dummies(df['publisher'])
publisher_df.head(5)

In [ ]:
# Summing the one-hot encoded publisher DataFrame to get the count of books per publisher
publisher_counts = publisher_df.sum().sort_values(ascending=False)
print(publisher_counts)

In [ ]:
outlier_rows = df[df['average_rating'] > 5]

# Display the rows with average_rating greater than 5
print("Rows with average_rating greater than 5:")
print(outlier_rows)

In [ ]:
# Get the top 10 publishers
top_n = 15
top_publishers = publisher_counts.head(top_n)

# Plotting the top N publishers
plt.figure(figsize=(12, 6))
top_publishers.plot(kind='bar')
plt.title(f'Top {top_n} Publishers by Number of Books')
plt.xlabel('Publisher')
plt.ylabel('Number of Books')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
author_df=pd.get_dummies(df['authors'])
author_df.head(5)

In [ ]:
df[df['authors']=='Donald A. Norman'].count()

# Standarization 

In [ ]:
features = pd.concat([rating_df, language_df, df['average_rating'],
                      df['ratings_count'], df['title'], publisher_df, author_df], axis=1)
features.set_index('title', inplace=True)

features.head()

In [ ]:
# Plot KDE for each numerical column
features.plot(kind='kde', subplots=True, layout=(3, 3), figsize=(15, 10), sharex=False)
plt.suptitle('KDE Plots of Features')
plt.show()

In [ ]:
# Separate continuous and categorical features
continuous_features = features[['average_rating', 'ratings_count']]
categorical_features = features.drop(columns=['average_rating', 'ratings_count'])

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Scale the continuous features only
scaler = StandardScaler()
continuous_features_scaled = scaler.fit_transform(continuous_features)

# Combine scaled continuous features with categorical features
# Note: categorical_features is a DataFrame, so we can concatenate it back
features_scaled = np.hstack((continuous_features_scaled, categorical_features.values))
print(features_scaled)


In [ ]:
features_scaled

# Model Building

In [ ]:
from sklearn import neighbors

In [ ]:
model = neighbors.NearestNeighbors(n_neighbors=30, algorithm='ball_tree', metric='euclidean')
model.fit(features_scaled)
dist, idlist = model.kneighbors(features_scaled)

In [ ]:
df['title'].value_counts()

In [ ]:
def BookRecommender(input_title):
    # Step 1: Check if the title exists in the DataFrame
    if input_title not in df['title'].values:
        print(f"Title '{input_title}' not found in the dataset.")
        return []
    
    # Step 2: Retrieve the index of the input title
    title_index = df[df['title'] == input_title].index[0]
    
    # Step 3: Create the feature vector for the input title
    feature_vector = features_scaled[title_index].reshape(1, -1)
    
    print("Feature Vector:", feature_vector)
    
    # Step 4: Find nearest neighbors
    distances, indices = model.kneighbors(feature_vector, n_neighbors=30)
    
    # Step 5: Retrieve recommended book titles based on indices
    recommended_books = []
    
    # Include the original book in recommendations
    original_book = df.iloc[title_index][['title', 'authors']].to_dict()
    recommended_books.append(original_book)

    for idx in indices[0]:
        if idx != title_index:  # Avoid recommending the same book again
            recommended_books.append(df.iloc[idx][['title', 'authors']].to_dict())
            if len(recommended_books) >= 30:  # Limit to 30 recommendations total
                break
    
    # Format output for all recommended books with their respective publishers
    formatted_recommendations = [
        f"{row['title']} by {row['authors']}" for row in recommended_books
    ]
    
    return formatted_recommendations

In [ ]:
def recommend_by_publisher(input_publisher):
    # Check if publisher exists in DataFrame columns
    if input_publisher not in features.columns:
        print(f"Publisher '{input_publisher}' not found in the dataset.")
        return []
    
    # Step 1: Retrieve books by the input publisher
    books_by_input_publisher = df[df['publisher'] == input_publisher]
    recommended_books = books_by_input_publisher[['title', 'authors', 'publisher']].copy()
    
    # Check if we have enough books
    if len(recommended_books) >= 30:
        # Format output for books from the input publisher
        return [f"{row['title']} by {row['authors']}, ({row['publisher']})" for _, row in recommended_books.iterrows()]
    
    # Step 2: If not enough books, find similar publishers
    feature_vector = np.zeros(features.shape[1])
    
    # Set the corresponding index for the input publisher to 1
    feature_vector[features.columns.get_loc(input_publisher)] = 1
    
    # Set average values for continuous features
    average_rating = df['average_rating'].mean()  
    ratings_count = df['ratings_count'].mean()  
    
    # Prepare an array with both continuous features for scaling
    continuous_features = np.array([[average_rating, ratings_count]])
    
    # Assign scaled values for continuous features
    scaled_values = scaler.transform(continuous_features)
    
    feature_vector[features.columns.get_loc('average_rating')] = scaled_values[0][0]
    feature_vector[features.columns.get_loc('ratings_count')] = scaled_values[0][1]
    
    # Reshape for model prediction
    feature_vector = feature_vector.reshape(1, -1)
    
    print("Feature Vector:", feature_vector)
    
    # Find nearest neighbors
    distances, indices = model.kneighbors(feature_vector, n_neighbors=30)
    
    # Step 3: Retrieve recommended book titles based on indices of similar publishers
    similar_publishers_indices = indices[0]
    
    # Get unique publishers from the similar publishers' indices
    similar_publishers = df.iloc[similar_publishers_indices]['publisher'].unique()
    
    additional_recommendations = []
    
    for publisher in similar_publishers:
        if publisher != input_publisher:  # Avoid recommending from the same publisher again
            additional_books = df[df['publisher'] == publisher][['title', 'authors', 'publisher']]
            additional_recommendations.extend(additional_books.to_dict(orient='records'))
            if len(additional_recommendations) >= (30 - len(recommended_books)):
                break
    
    # Combine recommendations
    all_recommended_books = recommended_books.to_dict(orient='records') + additional_recommendations
    
    # Format output for all recommended books with their respective publishers
    formatted_recommendations = [
        f"{row['title']} by {row['authors']}, ({row['publisher']})" for row in all_recommended_books
    ]
    
    # Limit to maximum of 25 recommendations and return formatted list
    return formatted_recommendations[:30]


In [ ]:
def recommend_by_author(input_author):
    # Step 1: Check if author exists in DataFrame without normalization
    if input_author not in df['authors'].values:
        print(f"Author '{input_author}' not found in the dataset.")
        return []
    
    # Step 2: Retrieve books by the input author
    books_by_input_author = df[df['authors'] == input_author]
    recommended_books = books_by_input_author[['title', 'authors']].copy()
    
    # Check if we have enough books
    if len(recommended_books) >= 30:
        # Format output for books from the input author
        return [f"{row['title']} by {row['authors']}" for _, row in recommended_books.iterrows()]
    
    # Step 3: If not enough books, find similar authors (without normalization)
    feature_vector = np.zeros(features.shape[1])
    
    # Attempt to get the feature index directly from input author name
    try:
        feature_index = features.columns.get_loc(input_author)
        feature_vector[feature_index] = 1
    except KeyError:
        print(f"Author '{input_author}' not found in features.")
        return []
    
    # Set average values for continuous features
    average_rating = df['average_rating'].mean()  
    ratings_count = df['ratings_count'].mean()  
    
    # Prepare an array with both continuous features for scaling
    continuous_features = np.array([[average_rating, ratings_count]])
    
    # Assign scaled values for continuous features
    scaled_values = scaler.transform(continuous_features)
    
    feature_vector[features.columns.get_loc('average_rating')] = scaled_values[0][0]
    feature_vector[features.columns.get_loc('ratings_count')] = scaled_values[0][1]
    
    # Reshape for model prediction
    feature_vector = feature_vector.reshape(1, -1)
    
    print("Feature Vector:", feature_vector)
    
    # Find nearest neighbors
    distances, indices = model.kneighbors(feature_vector, n_neighbors=30)
    
    # Step 4: Retrieve recommended book titles based on indices of similar authors
    similar_authors_indices = indices[0]
    
    additional_recommendations = []
    
    for idx in similar_authors_indices:
        similar_author = df.iloc[idx]['authors']
        
        if similar_author != input_author:  # Avoid recommending from the same author again
            additional_books = df[df['authors'] == similar_author][['title', 'authors']]
            additional_recommendations.extend(additional_books.to_dict(orient='records'))
            if len(additional_recommendations) >= (30 - len(recommended_books)):
                break
    
    # Combine recommendations
    all_recommended_books = recommended_books.to_dict(orient='records') + additional_recommendations
    
    # Format output for all recommended books with their respective publishers
    formatted_recommendations = [
        f"{row['title']} by {row['authors']}" for row in all_recommended_books
    ]
    
    # Limit to maximum of 25 recommendations and return formatted list
    return formatted_recommendations[:30]



In [ ]:
def recommend_books_by_rating(input_rating):
    # Filter books based on the input rating
    filtered_books = df[df['average_rating'] >= input_rating]
    
    # Sort the filtered books by average rating in descending order
    sorted_books = filtered_books.sort_values(by='average_rating', ascending=False)
    
    # Get the top 100 books or all available if fewer than 100
    recommended_books = sorted_books.head(100)
    
    # Format the output
    book_list_info = [f"{row['title']} by {row['authors']} " for index, row in recommended_books.iterrows()]
    
    return book_list_info
